<a href="https://colab.research.google.com/github/bimarshak7/finetuning-hf-models/blob/main/MT5_finetuning(Nepali_News_title_generation).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U transformers datasets accelerate SentencePiece -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00


In [2]:
import transformers
from datasets import load_dataset, load_metric, Dataset
from transformers import (
    T5Tokenizer,
    MT5ForConditionalGeneration,
    TrainingArguments,
    Trainer
)
import re
import string
import time
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
!gdown 1-71LzjRgPJblmNEs9t8XjmEU9qoh2CyW

Downloading...
From: https://drive.google.com/uc?id=1-71LzjRgPJblmNEs9t8XjmEU9qoh2CyW
To: /content/news_big.csv
100% 1.30G/1.30G [00:18<00:00, 70.8MB/s]


In [4]:
ds = Dataset.from_csv("news_big.csv").train_test_split(test_size=0.2)
ds = ds.remove_columns(["id", "date_published"])
ds

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['title', 'news'],
        num_rows: 181145
    })
    test: Dataset({
        features: ['title', 'news'],
        num_rows: 45287
    })
})

In [5]:
ds = ds.filter(lambda eg: (len(eg['news'].split()) >= 1000))
ds

Filter:   0%|          | 0/181145 [00:00<?, ? examples/s]

Filter:   0%|          | 0/45287 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['title', 'news'],
        num_rows: 10410
    })
    test: Dataset({
        features: ['title', 'news'],
        num_rows: 2657
    })
})

In [6]:
ds_train = ds["train"]
ds_test = ds["test"]

In [7]:
MODEL = "google/mt5-base"

In [8]:
!ls /.cache

ls: cannot access '/.cache': No such file or directory


In [9]:
tokenizer = T5Tokenizer.from_pretrained(MODEL)
tokenizer

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


T5Tokenizer(name_or_path='google/mt5-base', vocab_size=250100, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [10]:
!ls /home

In [11]:
stop_words = stopwords.words('nepali')

In [12]:
def clean_text(eg,isNews=True):
    #remove punctuations
    text = eg["news"]
    title = eg["title"]

    sentences = text.replace("\n", " ").split(u"।")
    sentences_title = title.replace("\n", " ").split(u"।")

    f = [sentence.translate(str.maketrans('', '', string.punctuation)) for sentence in sentences]
    f_title = [sentence.translate(str.maketrans('', '', string.punctuation)) for sentence in sentences_title]

    f = [x.strip() for x in f if len(x)>0]
    f_title = [x.strip() for x in f_title if len(x)>0]

    if isNews:
        f = [word for words in f for word in words.split() if word not in stop_words]
    text =  " ".join(f)
    text = re.sub(r"\d", "", text)

    title =  " ".join(f_title)
    title = re.sub(r"\d", "", title)

    eg["title"] = title
    eg["news"] = text

    return eg

In [13]:
ds_train = ds_train.map(clean_text)
ds_test = ds_test.map(clean_text)

Map:   0%|          | 0/10410 [00:00<?, ? examples/s]

Map:   0%|          | 0/2657 [00:00<?, ? examples/s]

In [14]:
max_input_length = 1000
max_target_length = 28

In [15]:
def preprocess_data(examples):
  model_inputs = tokenizer(examples["news"], max_length=max_input_length, truncation=True)

  # Setup the tokenizer for targets
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples["title"], max_length=max_target_length, truncation=True)

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [16]:
train_prep = ds_train.map(preprocess_data, batched=True)
test_prep = ds_test.map(preprocess_data, batched=True)

Map:   0%|          | 0/10410 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3860: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2657 [00:00<?, ? examples/s]

In [17]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [18]:
batch_size = 4
model_dir = f"{MODEL}"

In [29]:
args = Seq2SeqTrainingArguments(
    model_dir,
    evaluation_strategy="steps",
    eval_steps=1000,
    logging_strategy="steps",
    logging_steps=1000,
    save_strategy="steps",
    save_steps=1000,
    learning_rate=4e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    report_to=None
)

In [20]:
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [21]:
!pip install rouge_score -q

  Preparing metadata (setup.py) ... done


In [22]:
metric = load_metric("rouge")

<ipython-input-22-7154407f1f07>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [23]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip()))
                      for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip()))
                      for label in decoded_labels]

    # Compute ROUGE scores
    result = metric.compute(predictions=decoded_preds, references=decoded_labels,
                            use_stemmer=True)

    # Extract ROUGE f1 scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length to metrics
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id)
                      for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [24]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [25]:
import gc
gc.collect()

248

In [30]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=train_prep,
    eval_dataset=test_prep,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [31]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1000,0.000000,nan,0.000000,0.000000,0.000000,0.000000,13.211100


In [ ]:
news = """
प्रतिनिधिसभा सदस्य तथा पूर्व मन्त्री दीपक खड्का चढेको गाडी धनकुटामा दुर्घटना हुँदा धरान–धनकुटा सडक अवरुद्ध भएको छ । खड्का सवार स्कोरपियो गाडी टिपरसँग ठोकिएर सडकमै थन्किएपछि सडक अवरुद्ध भएको हो । अर्को पनि दुर्घटना एकै ठाउँमा हुँदा तीन वटा गाडी सडकमै रहेकाले सडक अवरुद्ध हुँदा हजारौ यात्रु अलपत्र परेका छन् ।
सङ्खुवासभाको चैनपुरबाट विराटनगर जाँदै गरेको प्रदेश नं.१–०२–००२ च ३५४५ नम्बरको स्कर्पियो गाडी धनकुटाको साँगुरी गढी गाउँपालिकाको कोल बोटेमा इटहरीबाट धनकुटा आउँदै गरेको प्रदेश नं.२–०३–००१ क १५२१ नम्बरको टिपरसँग ठोकिएको जिल्ला प्रहरी कार्यालय धनकुटाका प्रहरी प्रमुख कृष्ण कोइरालाले बताउनुभयो ।
सडकबाट दुर्घटनाग्रस्त गाडी र ट्रक निकाल्न समस्या भएका कारण सडक अवरुद्ध भएको प्रहरीले जनाएको छ ।"""

In [ ]:
news_clean = clean_text({"news":news,"title":""})

In [ ]:
inputs = tokenizer(news_clean["news"], max_length=max_input_length, truncation=True, return_tensors="pt").to("cuda")

In [ ]:
output = model.generate(**inputs, num_beams=10, do_sample=True, min_length=10, max_length=64)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)

print(predicted_title)